# Pandas

Mit Pandas (= panel data) lassen sich tabellarische Daten sehr komfortabel handhaben. Mehr unter https://pandas.pydata.org/.

Sehr hilfrich:
- https://pandas.pydata.org/docs/getting_started/intro_tutorials/index.html
- https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

Pandas nutzt zwei wichtige Datenstrukturen:
- `Series` sind 1-dimensionale Arrays. Im Gegensatz zu Numpy-Arrays haben sie aber einen zusätzlichen Index, über den man komfortabel auf Einträge zugreifen kann. Intern baut dies auf Numpy-Arrays auf.
- `DataFrame` ist eine ganze Tabelle bestehend aus mehreren Series (=Spalten) gleicher Länge.

In [2]:
import pandas as pd
th = pd.read_csv("TH-Aschaffenburg.csv")
th.head(10)

,Jahr,Fach,Anfänger,Studierende
0,WS 02/03,BW,112.0,430.0
1,WS 02/03,BWR,57.0,230.0
2,WS 02/03,EIT,50.0,178.0
3,WS 02/03,M,44.0,44.0
4,WS 02/03,WI,69.0,183.0
5,WS 03/04,BW,116.0,460.0
6,WS 03/04,BWR,61.0,243.0
7,WS 03/04,EIT,57.0,198.0
8,WS 03/04,M,44.0,81.0
9,WS 03/04,WI,75.0,219.0


#### DataFrames

Jede Spalte ist eine Series.

Die vorderste Spalte eines DataFrame (oder Series) ist der sogenannte Index. Er enthält die *Zeilennamen*. Standardmäßig sind dies die Zahlen 0, 1, 2, ...
Man kann jedoch auch einen anderen Index setzen.

Man kann eine Series auch explizit erzeugen. Falls man keinen Index angibt, sind es wieder die Zahlen 0, 1, 2, ...

#### Daten einlesen und schreiben

Pandas bietet eine Vielzahl an Möglichkeiten um Daten aus Datenbanken (via SQL) oder aus Dateien einzulesen, z.B. csv, xlsx, parquet, feather, ...

Ersten Einblick in die Daten gewinnen.

Auch das Lesen und Schreiben von xlsx-Dateien ist kein Problem. Hier exemplarisch den Datensatz als xlsx abspreichern. Hierfür muss das Paket *openpyxl* installiert sein.

#### Auswahl von Spalten oder Zeilen

Wählt man nicht nur einen Spaltennamen sondern eine Liste von mehreren Spalten, so erhält man einen DataFrame.

Bei der Zeilenauswahl möchte man meistens Zeilen selektieren, die bestimmte Eigenschaften erfüllen.
Hierfür schreibt man in die eckigen Klammern `[]` einen True/False-Vektor von der Länge des DataFrames.
Oft sind Vergleichsoperatoren (`<`, `>`, `==`, `<=`, `>=`, `!=`) und Boole'sche Operatoren `&` (and), `|` (or) und `~` (not) sehr hilfreich.

Möchte man *in einem Schritt* gewisse Zeilen und Spalten auswählen, eignen sich `.loc` und `.iloc`.


Mittels `.loc[rows, columns]` (= location) kann man Einträge über ihren **Zeilenindex** ansprechen.

Mittels `.iloc[row, column]` (= integer location) kann man Einträge über ihre Zeilen-/Spalten**nummer**, d.h. über die Position und nicht über den Namen, auswählen. (Hier funktioniert das Slicing wie gewohnt.)

Mit `.reset_index` lässt sich der Index zu einer normalen Spalte machen. 

#### Spalten erzeugen, entfernen und umbenennen

Beim berechnen neuer Spalten wird die Mächtigkeit der Vektorisierung deutlich.
Die Berechnungen finden elementweise für jeden Eintrag des Vektors statt.
Eine explizite Schleife ist nicht nötig.

Möchte man komplizierte Berechnungen durchführen, kann man eine Funktion schreiben und diese mittels `map()` oder `apply()` zeilenweise 
anwenden. Mehr dazu später! 

Zur Umbenennung von Spalten übergibt man der Funktion `.rename` ein Dictionary mit Einträgen der Form `{"old_colname": "new_colname"}` oder eine geeignete String-Funktion.

#### Visualisierung

(Hier nur ganz kurz. Visualisierung bekommt noch eine Extrasession.)

#### Sortieren

Die Sortierung hat natürlich Konsequenzen für den Index. 
Möchte man diesen wieder korrigieren, so kann man entweder den Index reseten oder ihn direkt bei der Sortierung ignorieren lassen. 

#### Aggregation von Daten

Typische Aggregationsfunktionen wie z.B. mean(), sum(), max(), quantile() ... können direkt als Methode von `Series` aufgerufen werden.

Sehr oft möchte man derartige Aggregationen gruppenweise, d.h. für jede Ausprägung eines Merkmals, durchführen. Dies ist bekannt als **Split-Apply-Combine**: Die Gesamttabelle wird gemäß der Ausprägungen eines Merkmals in Einzeltabellen geteilt, für jede solche Tabelle berechnet man die Aggregationsfunktion und anschließend werden diese Werte in einer kleineren Tabelle gesammelt.

Soeben haben wir direkt nach `.groupby()` eine eckige Klammer mit Spaltennamen genutzt. Dies wird oft gemacht und schränkt die Einzeltabellen auf die genannten Spalten ein. Die Aggregationsfunktion wirkt dann nur noch auf diese Spalten.

Man kann auch nach mehreren Merkmalen gruppieren. Das resultierende Objekt hat dann einen MultiIndex.

Sehr oft möchte man für jede Ausprägung eines Merkmals auszählen wie oft es vorkommt. Dies liefert die Grundlage für Balkendiagramme.
(Achtung: Standardmäßig bilden NA-Werte keine eigene Gruppe und werden nicht aufgeführt. Dies ist im Rahmen der Datenaufbereitung allerdings oft relevant. Hierfür kann man in `.groupby()` oder in `.value_counts()` die Option `dropna=False` setzen.)
Da das resultierende Objekt kein DataFrame ist, eignet sich ein `.reset_index()` um den Index zu einer expliziten Spalte zu machen und z.B. Sortierschritte anzuschließen.

Es gibt sowohl `.count()` als auch `.size()`. Diese sind sehr ähnlich. Während `.size()` die Zeilenanzahl zurückgibt, liefert `.count()` die Anzahl an Nicht-NA-Werten. Die eine Funktion zählt also NA-Werte mit, die andere hingegen nicht.

#### Datenschubsen für Fortgeschrittene: Long-Format und Wide-Format

<!--- Datenquelle:
- Destatis, Tabelle 12411-0015: Bevölkerung: Kreise, Stichtag (Auswahl: alle Stichtage). (Download Flat-File!)
- Destatis, Tabelle 12411-0018: Bevölkerung: Kreise, Stichtag, Geschlecht, Altersgruppen (Auswahl: alle Stichtage aber nur die 3 Kreise Aschaffenburg, Würzburg und Schweinfurt). (Download Flat-File!) -->

Es gibt verschiedene Möglichkeiten ein und die selben Daten tabellarisch darzustellen.
Je nach Anwendungsfall ist es nötig zwischen diesen Formen zu wechseln. Dies ist manchmal als "pivotieren" bekannt.
Am besten sieht man es an einem Beispiel. 

<!--- Die nachfolgende Tabelle (adaptiert nach Destatis Tabelle 12411-0016) stellt die Bevölkerungsentwicklung der drei Städte Aschaffenburg, Schweinfurt und Würzburg dar.-->

Die nachfolgende Tabelle enthält die Entwicklung der Studierendenzahlen für die TH Aschaffenburg

In [ ]:
import pandas as pd
th = pd.read_csv("TH-Aschaffenburg.csv")
th.head(10)

<!--- Zur Einfachheit vernachlässigen wir zunächst die Geschlechtsunterscheidung und beschränken uns auf die Gesamtzahlen.
Die Tabelle ist im Long-Format und enthält für jede Kombination aus Ort und Zeit eine Zeile mit der Bevölkerungszahl.
Dies hat den Vorteil, dass problemlos weitere Orte und Zeiten ergänzt werden können ohne die Struktur der Tabelle ändern zu müssen.-->

Zur Einfachheit vernachlässigen wir zunächst die Gesamtstudierendenzahlen und beschränken uns nur auf die Studiengänge *BW* (Betriebswirtschaft), *SD* (Software Design) und *EIT* (Elektro- und Informationstechnik) seit dem Wintersemester 2020.

In [10]:
th_small = (th[th["Fach"].isin(["BW","EIT","SD"]) & th["Jahr"].isin(["WS 22/23", "WS 21/22", "WS 20/21"]) ]
    .drop(columns = "Studierende")).reset_index(drop=True)
th_small


,Jahr,Fach,Anfänger
0,WS 20/21,BW,157.0
1,WS 20/21,EIT,29.0
2,WS 20/21,SD,55.0
3,WS 21/22,BW,113.0
4,WS 21/22,EIT,28.0
5,WS 21/22,SD,52.0
6,WS 22/23,BW,104.0
7,WS 22/23,EIT,34.0
8,WS 22/23,SD,64.0


Die obige Tabelle beinhaltet für jede Kombination von Studienjahr und Fach eine Zeile. 
Dies nennt man **Long-Format**.
Insbesondere für Zeitreihen ist es jedoch oft übersichtlicher die Daten anders anzuordnen.
Hier wollen wir die Anfängerzahlen jedes Faches in einer separaten Spalte darstellen.
Dies ist dann das **Wide-Format**.

Mit dem Befehl `pivot()` kann man die Felder geeignet "rotieren".
Hierbei muss man angeben welche Spalten als Index erhalten bleiben sollen, welche Spalte die neuen Spaltennamen enthält und welches die eigentlichen Werte sind.
Das Wide-Format mag übersichtlicher erscheinen, hat jedoch den Nachteil, dass man das Tabellenschema ändern muss wenn neue Studiengänge hinzukommen.

In [19]:
th_small_wide =th_small.pivot(index="Jahr", columns="Fach", values ="Anfänger").reset_index()
th_small_wide
#th_small.pivot(index="Fach", columns="Jahr", values ="Anfänger")

Fach,Jahr,BW,EIT,SD
0,WS 20/21,157.0,29.0,55.0
1,WS 21/22,113.0,28.0,52.0
2,WS 22/23,104.0,34.0,64.0


Natürlich kann man auch vom Wide-Format zum Long-Format konvertieren.
Dies funktionert mit `melt()`.
(Zunächst machen wir jedoch mit `reset_index()` den Index zur einer regulären Spalte.)
Im Argument `id_vars` listet man alle Spalten, die konstant gehalten werden sollen - alle anderen Spalten werden zu zwei neuen Spalten "pivotiert", indem jede Kombination aus Spaltenname und jeweilem Eintrag eine neue Zeile bilden. 

In [21]:
th_small_wide.melt(id_vars = "Jahr")

,Jahr,Fach,value
0,WS 20/21,BW,157.0
1,WS 21/22,BW,113.0
2,WS 22/23,BW,104.0
3,WS 20/21,EIT,29.0
4,WS 21/22,EIT,28.0
5,WS 22/23,EIT,34.0
6,WS 20/21,SD,55.0
7,WS 21/22,SD,52.0
8,WS 22/23,SD,64.0


Soeben hatten wir den Datensatz noch auf die Anfängerzahlen eingeschränkt und die Spalte *Studierende* entfernt.
So haben wir `pivot()` nur die Spalte *Anfänger* pivotieren lassen.

Belässt man hingegen die Studienrendenzahlen im Datensatz, so können wir auch diese mitpivotieren.
Hierbei entsteht für die Spalten ein **MultiIndex**.

Eng verwandt mit `pivot()` und `melt()` sind die Funktionen `stack()` und `unstack()`.
Sie sind vor allem im Zusammenspiel mit Indexen und MultiIndexen sinnvoll.

- `stack()` pivotiert ("stapelt") alle Spaltenlabel (außer dem Index) und liefert eine Series (oder einen DataFrame) mit einem zusätzlichen inneren Indexlevel.
- `unstack()` pivotiert das innerste Indexlevel zu Spaltenlabeln und erzeugt so einen DataFrame.

Beschränken wir uns zur Übersichtlichkeit auf die obigen Anfängerzahlen.

`stack()` rotiert alle Spaltennamen zu einem neuen inneren Indexlevel und liefert somit eine Series mit einem zweistufigen Multiindex.

Mittels `unstack()` wird das innerste Level eines MultiIndex zu neuen Spaltennamen und wir erhalten wieder den ursprünglichen DataFrame.
(Möchte man anstatt des innersten Levels ein anderes Indexlevel pivotieren, so kann man sowohl `stack` als auch `unstack` ein Indexlevel angeben.)

Nebenbei: Anstatt `pivot` aufzurufen kann man auch mittels `set_index` einen MultiIndex erzeugen und im Anschluss `unstack` aufrufen.

#### Zusammenführen mehrerer Tabellen

Sehr oft möchte man mehrere Tabellen kombinieren um Informationen anzureichern.
Auch mit Pandas kann man mehrere DataFrames verjoinen, so wie man es von SQL kennt.

Die einfachste Variante um zwei DataFrames zu kombinieren ist das simple Untereinanderhängen.
Dies ergibt selbstverständlich nur dann Sinn, wenn die Spalten übereinstimmen (oder zumindest Teilmengen voneinander sind).
Betrachten wir hier beispielsweise die Daten zum Fach *MEDS* sowie zu *BW KMU*.
Mit `concat()` lassen sich die beiden Tabellen direkt hintereinanderhängen.

In [26]:
df_1 = th[th["Fach"] == "MEDS"]
df_2 = th[th["Fach"] == "SD"]
pd.concat([df_1, df_2])

df_1 = th[th["Fach"] == "MEDS"]
df_2 = th[th["Fach"] == "SD"].drop(columns = "Anfänger")
pd.concat([df_1, df_2])

,Jahr,Fach,Anfänger,Studierende
184,WS 19/20,MEDS,48.0,48.0
204,WS 20/21,MEDS,31.0,56.0
225,WS 21/22,MEDS,20.0,63.0
250,WS 22/23,MEDS,19.0,78.0
210,WS 20/21,SD,NaN,55.0
232,WS 21/22,SD,NaN,90.0
258,WS 22/23,SD,NaN,138.0


Inhaltlich spannender ist das Verjoinen von Tabellen um zusätzliche Spalten zu erhalten.
In der bisherigen Tabelle sind z.B. nur die Studiengangskürzel enthalten und auch eine Zuordnung zu Fakultäten fehlt.
Diese Information liefert die folgende Tabelle

In [27]:
th_names = pd.read_csv("TH-Aschaffenburg_Faecher.csv")
th_names.head()

,Abkürzung,Langname,Fakultät
0,BW,Betriebswirtschaft,WR
1,BW KMU,Betriebswirtschaft für KMU,WR
2,BWR,Betriebswirtschaft und Recht,WR
3,DIM,Digitales Immobilienmanagement,WR
4,IIM,Internationales Immobilienmanagement,WR


Mittels `merge()` kann man in Pandas alle Arten von JOIN-Operationen durchführen, wie man sie aus SQL kennt.
Hier wollen wir die Tabelle *th_small* mit den jeweiligen Studiengangsinformationen anreichern.

In [29]:
pd.merge(left = th_small, right = th_names, how = "left", left_on = "Fach", right_on = "Abkürzung")

,Jahr,Fach,Anfänger,Abkürzung,Langname,Fakultät
0,WS 20/21,BW,157.0,BW,Betriebswirtschaft,WR
1,WS 20/21,EIT,29.0,EIT,Elektro- und Informationstechnik,IW
2,WS 20/21,SD,55.0,SD,Software Design,IW
3,WS 21/22,BW,113.0,BW,Betriebswirtschaft,WR
4,WS 21/22,EIT,28.0,EIT,Elektro- und Informationstechnik,IW
5,WS 21/22,SD,52.0,SD,Software Design,IW
6,WS 22/23,BW,104.0,BW,Betriebswirtschaft,WR
7,WS 22/23,EIT,34.0,EIT,Elektro- und Informationstechnik,IW
8,WS 22/23,SD,64.0,SD,Software Design,IW


Mit `merge()` lassen sich auch andere Joins durchführen, beispielsweise ein RIGHT JOIN (aber auch INNER JOIN oder OUTER JOIN).